In [3]:
import tensorflow as tf
from tensorflow import keras

In [4]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [5]:
!pip install -q -U keras-tuner

In [6]:
import keras_tuner as kt

In [7]:
(img_train,label_train),(img_test,label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [8]:
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0

In [12]:
#zdefiniowanie modelu
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))
    hp_units = hp.Int('units',min_value=32,max_value=512,step=32)
    model.add(keras.layers.Dense(units=hp_units,activation='relu'))
    model.add(keras.layers.Dense(10))

    hp_learning_rate = hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4,2.88e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model


In [13]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_kt')

In [14]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [15]:
#wyszukiwanie hiperparametrów
tuner.search(img_train,label_train,epochs=50,validation_split=0.2,callbacks=[stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Wyszukiwanie hiperparametrów zakończone.
Optymalna liczba nueronów warstwy ukrytej: {best_hps.get('units')}.
Optymalna wartośc LR: {best_hps.get('learning_rate')}.

""")

Trial 30 Complete [00h 01m 25s]
val_accuracy: 0.8840833306312561

Best val_accuracy So Far: 0.8928333520889282
Total elapsed time: 00h 14m 57s

Wyszukiwanie hiperparametrów zakończone.
Optymalna liczba nueronów warstwy ukrytej: 416.
Optymalna wartośc LR: 0.000288.




In [16]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train,label_train,epochs=50,validation_split=0.2)

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5379 - accuracy: 0.8154 - val_loss: 0.4534 - val_accuracy: 0.8407
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3973 - accuracy: 0.8599 - val_loss: 0.3788 - val_accuracy: 0.8640
Epoch 3/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3558 - accuracy: 0.8732 - val_loss: 0.3546 - val_accuracy: 0.8743
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3277 - accuracy: 0.8817 - val_loss: 0.3516 - val_accuracy: 0.8726
Epoch 5/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3081 - accuracy: 0.8888 - val_loss: 0.3379 - val_accuracy: 0.8788
Epoch 6/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2895 - accuracy: 0.8935 - val_loss: 0.3175 - val_accuracy: 0.8872
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2753 - accuracy: 0.8994 - val_loss: 0.3393 - val_accuracy:

In [17]:
val_acc_per_epoch = history.history['val_accuracy']
best_epochs = val_acc_per_epoch.index(max(val_acc_per_epoch))+1
print(f'najlepsza epoka: {(best_epochs,)}')

najlepsza epoka: (42,)


In [18]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(img_train,label_train,epochs=best_epochs,validation_split=0.2)

Epoch 1/42
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5388 - accuracy: 0.8159 - val_loss: 0.4390 - val_accuracy: 0.8481
Epoch 2/42
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3995 - accuracy: 0.8582 - val_loss: 0.4022 - val_accuracy: 0.8560
Epoch 3/42
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3569 - accuracy: 0.8733 - val_loss: 0.3594 - val_accuracy: 0.8748
Epoch 4/42
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3285 - accuracy: 0.8817 - val_loss: 0.3500 - val_accuracy: 0.8758
Epoch 5/42
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3065 - accuracy: 0.8883 - val_loss: 0.3409 - val_accuracy: 0.8783
Epoch 6/42
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2905 - accuracy: 0.8947 - val_loss: 0.3281 - val_accuracy: 0.8815
Epoch 7/42
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2751 - accuracy: 0.8987 - val_loss: 0.3118 - val_accuracy:

In [19]:
eval_hypermodel = hypermodel.evaluate(img_test,label_test)
print(f'[test loss, test accuracy]: {eval_hypermodel}')

313/313 [==============================] - 1s 2ms/step - loss: 0.4042 - accuracy: 0.8886
[test loss, test accuracy]: [0.40421128273010254, 0.8885999917984009]
